# Tensorflow for eigenvalue problem

Testing audograd for eigenvalue problem. Or PPM. 

Suppose True problem have eigenvalues $e_t(c)$.
Prepare random matrix
$M(c) = M_0 + c M_1$, and solve eigenvalue problem $M(c) v = e v$. 

The goal is to reduce the difference $|e(c)-e_t(c)|^2$
by updating matrix $M_0$ and $M_1$. In other words, obtaining 
$M(c)$ which simulate the original problem. 


In [1]:
#========== import packages=============== 
import numpy as np
import tensorflow as tf
#tf.get_logger().setLevel('ERROR')
#tf.autograph.set_verbosity(1)

In [62]:
#============set up matrix====================
# target true matrix
c= tf.constant(0.5,dtype=tf.complex128)
s0 = tf.constant( [ [1.0,0.0],[0.0,1.0]],dtype=tf.complex128)
sx = tf.constant( [ [0.0,1.0],[1.0,0.0]],dtype=tf.complex128)
sy = tf.constant( [ [0.0+0j,-1j],[1j,0.0+0j]],dtype=tf.complex128)
sz = tf.constant( [ [1.0,0.0],[0.0,-1.0]],dtype=tf.complex128)

def mm_t(c):
    return sx+ sz*c 

e_t,v_t = tf.linalg.eig(mm_t(0.5))

#------initial matrix
learning_rate=tf.constant(0.01,dtype=tf.complex128)
var = tf.Variable([0.1,0.5,0.1,1.0],dtype=tf.complex128) # true is (0,1,0,0.5)

def mm_var(var):
    return s0*var[0]+sx*var[1]+sy*var[2]+sz*var[3]

e,v = tf.linalg.eig(mm_var(var))

print('mm_t :{} '.format(mm_t(0.5)))
print('e_t :{} '.format(e_t))
print('mm(0) :{} '.format(mm_var(var)))
print('e(0) :{} '.format(e))


mm_t :[[ 0.5+0.j  1. +0.j]
 [ 1. +0.j -0.5+0.j]] 
e_t :[ 1.11803399+0.j -1.11803399+0.j] 
mm(0) :[[ 1.1+0.j   0.5-0.1j]
 [ 0.5+0.1j -0.9+0.j ]] 
e(0) :[-1.02249722+6.30407896e-18j  1.22249722-6.30407896e-18j] 


In [66]:
#----compute loss and record tape 
def one_step(var):
    with tf.GradientTape() as tape:
        #print('mat:\n {}'.format(mm.numpy()))     
        e,v = tf.linalg.eig(mm_var(var))
        #print('e(0) :{} '.format(e))
        loss = tf.math.reduce_sum(tf.math.abs(e-e_t)**2)
        print('loss: {}'.format(loss.numpy()))     
        #---update
    dvar = tape.gradient(loss,var)
    var = tf.Variable(var - dvar*learning_rate,dtype=tf.complex128)
    return var 

In [70]:
for i in range(10):
    var = one_step(var)

loss: 2.630531168888958
loss: 2.503392453776252
loss: 2.754513061882956
loss: 2.5354459446721584
loss: 2.68046668289087
loss: 2.5380149029078
loss: 2.674045085053906
loss: 2.542789635007505
loss: 2.6681808292186777
loss: 2.5472393864529987


In [56]:
e,v = tf.linalg.eig(mm_var(var))
print(var)
print(e)
print(e_t)

<tf.Variable 'Variable:0' shape=(4,) dtype=complex128, numpy=
array([ 0.00049575-2.27464719e-18j,  0.00385201+3.49018562e-03j,
       -0.01937112-1.75515705e-02j,  0.00085826+7.77644580e-04j])>
tf.Tensor([-0.01927329-0.01791211j  0.02026478+0.01791211j], shape=(2,), dtype=complex128)
tf.Tensor([ 1.11803399+0.j -1.11803399+0.j], shape=(2,), dtype=complex128)


#----this is not working-----------------------
def CreateModel():
    X_input = tf.keras.layers.Input(1)
    var1 = tf.Variable(tf.random.uniform([4]))
    var2 = tf.Variable(tf.random.uniform([4]))
    mm = mm_var(var1)+X_input*mm_var(var2)
    e,v = tf.linalg.eig(mm)
    X_output = e
    model = tf.keras.Model(inputs = X_input,outputs=X_output)
    return model 

# PyTorch test

In [120]:
#-----testing pytorch
import torch
from torch import FloatTensor
from torch.autograd import Variable

w = Variable(FloatTensor([[4,2],[2,1]]),requires_grad=True)
ee,vv = torch.linalg.eig(w)
loss = torch.abs(ee-torch.Tensor([ 1.0,-1.0])).sum()
print(loss)
loss.backward()
dw = w.grad

tensor(5., grad_fn=<SumBackward0>)


In [168]:
for i in range(1):
    w =  Variable(w-dw*0.01,requires_grad=True) 
    ee,vv = torch.linalg.eig(w)
    loss = torch.abs(ee-torch.Tensor([ 1.0,-1.0])).sum()
    print(loss)
    loss.backward()
    dw = w.grad

tensor(5.7220e-06, grad_fn=<SumBackward0>)


In [169]:
ee

tensor([ 1.0000+0.j, -1.0000+0.j], grad_fn=<LinalgEigBackward0>)